# Livable Streets: Bike Infrastructure & Displacement
## Project Deliverable 1

#### Siddharth Bookinkere, Alexander Heger, Kwan Wing Tuet, Raviv Zait

## Clone repository

In [1]:
!git clone https://github.com/BU-Spark/ds-livable-streets-infra.git

Cloning into 'ds-livable-streets-infra'...


## Collect and Pre-process Census and Bike Data

In [4]:
import pandas as pd
import numpy as np
import os

# Set the current working directory to the root of the repository
os.chdir("C:/Users/Alex H/OneDrive/Documents/BU Fall 2023/DS701/ds-livable-streets-infra/")

# Customize data folder
data_folder = "fa23-team/data/"

# Read demographic and property value data
combined_demo_prop = pd.read_csv(data_folder + "combined_demo_prop.csv", index_col=False)
print(combined_demo_prop.shape)

# Read income and house ownership data
combined_income_owner = pd.read_csv(data_folder+"income_houseownership.csv", index_col=False)
print(combined_income_owner.shape)

# Read bike infrastructure data
bike_data = pd.read_csv(data_folder + "analysis/roxbury_dorchester_bike_data.csv", index_col=False)

(564, 16)